
Inspired by Quix's [simple-kafka-python](https://github.com/quixio/simple-kafka-python/tree/main).

In [28]:
from dotenv import load_dotenv

load_dotenv() #load from your .env file
app_key = os.getenv('TFL_API_KEY')

line_id = "victoria"
# stop_point_id = get_stop_point_id("Brixton")
direction = "outbound"

In [29]:
import os
def get_stop_point_id(stop_point_name):
    response = requests.get(
        "https://api.tfl.gov.uk/StopPoint/Search/" + stop_point_name,
        params = {
            'app_key': app_key
        }
    )
    print(response)
    data = response.json()
    print(data)

get_stop_point_id("Brixton")

<Response [200]>
{'$type': 'Tfl.Api.Presentation.Entities.SearchResponse, Tfl.Api.Presentation.Entities', 'query': 'Brixton', 'total': 8, 'matches': [{'$type': 'Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities', 'icsId': '1000031', 'modes': ['national-rail', 'bus', 'tube'], 'zone': '2', 'id': 'HUBBRX', 'name': 'Brixton', 'lat': 51.462961, 'lon': -0.114531}, {'$type': 'Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities', 'icsId': '1004362', 'topMostParentId': '490G00004362', 'modes': ['bus'], 'id': '490G00004362', 'name': 'Brixton Water Lane', 'lat': 51.456341, 'lon': -0.118328}, {'$type': 'Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities', 'icsId': '1007526', 'topMostParentId': '490G00007526', 'modes': ['bus'], 'id': '490G00007526', 'name': 'Brixton Academy', 'lat': 51.465238, 'lon': -0.115183}, {'$type': 'Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities', 'icsId': '1008544', 'topMostParentId'

# Main functions below, above is testing it coming together

## API docs
https://content.tfl.gov.uk/trackernet-data-services-guide-beta.pdf older one I think  
https://api-portal.tfl.gov.uk/api-details#api=Line&operation=Line_ArrivalsByPathIds&definition=Tfl-41 from portal

In [ ]:
import requests
import time
import json


def get_stop_point_id(stop_point_name):
    response = requests.get(
        "https://api.tfl.gov.uk/StopPoint/Search/" + stop_point_name,
        params = {
            'app_key': app_key
        }
    )
    data = response.json()
    return data
    # Extract the stop point ID from the response
    # if 'matches' in data and len(data['matches']) > 0:
    #     return data['matches'][0]['id']
    # else:
    #     return None


def get_tube_arrivals(line_id, stop_point_id, direction, app_key):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
    response = requests.get(
        url,
        params = {
            'direction': direction,
            'app_key': app_key,
        }
    )
    return response.json()

In [ ]:
from dotenv import load_dotenv

load_dotenv() #load from your .env file
app_key = os.getenv('TFL_API_KEY')

line_id = "victoria"
stop_point_id = get_stop_point_id("Brixton")
direction = "outbound"

In [ ]:
import logging
from quixstreams import Application

def main():
    app = Application(
        broker_address="localhost:19092",
        loglevel="DEBUG",
    )

    with app.get_producer() as producer:
        while True:
            weather = get_weather()
            logging.debug("Got weather: %s", weather)
            producer.produce(
                topic="weather_data_demo",
                key="London",
                value=json.dumps(weather),
                headers= {"app.name": "python-quix"}
            )

            weather_moscow=get_weather_moscow()
            logging.debug("Got weather: %s", weather_moscow)
            producer.produce(
                topic="weather_data_demo",
                key = "Moscow",
                value=json.dumps(weather_moscow),
                headers= {"app.name": "python-quix"}
            )
            logging.info("Produced. Sleeping...")
            time.sleep(5)


if __name__ == "__main__":
    logging.basicConfig(level="DEBUG")
    main()